<a href="https://colab.research.google.com/github/NailaHamad/Django/blob/master/AHPTOPSIS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install numpy pandas

In [2]:
import numpy as np
import pandas as pd

In [3]:
# Saaty pairwise comparison matrix (example values)
AHP_matrix = np.array([
    [1,   1/3, 3,   5],
    [3,   1,   5,   7],
    [1/3, 1/5, 1,   3],
    [1/5, 1/7, 1/3, 1]
])

# Normalize columns
col_sum = AHP_matrix.sum(axis=0)
norm_matrix = AHP_matrix / col_sum

# Compute priority vector (weights)
weights = norm_matrix.mean(axis=1)
weights = weights / weights.sum()      # ensure they sum to 1

criteria = ["Cost", "Flexibility", "Usability", "UserFriendliness"]
pd.DataFrame({"Criteria": criteria, "Weight": weights})


,Criteria,Weight
0,Cost,0.263345
1,Flexibility,0.557892
2,Usability,0.121873
3,UserFriendliness,0.056890


In [4]:
data = {
    "Cost":              [7, 9, 8, 6],  # AWS, Google, Azure, IBM
    "Flexibility":       [9, 8, 7, 6],
    "Usability":         [8, 9, 8, 7],
    "UserFriendliness":  [7, 9, 8, 8],
}
alternatives = ["AWS", "Google", "Azure", "IBM"]
df = pd.DataFrame(data, index=alternatives)
df


,Cost,Flexibility,Usability,UserFriendliness
AWS,7,9,8,7
Google,9,8,9,9
Azure,8,7,8,8
IBM,6,6,7,8


In [5]:
# Step 1: Normalize the decision matrix
norm = df / np.sqrt((df**2).sum())

# Step 2: Weighted normalized matrix
weighted = norm * weights

# Step 3: Determine ideal best and worst
ideal_best  = weighted.max()
ideal_worst = weighted.min()

# Step 4: Compute distances
dist_best  = np.sqrt(((weighted - ideal_best)**2).sum(axis=1))
dist_worst = np.sqrt(((weighted - ideal_worst)**2).sum(axis=1))

# Step 5: Relative closeness to ideal
performance_score = dist_worst / (dist_best + dist_worst)

# Step 6: Rank
df_results = pd.DataFrame({
    "S+": dist_best,
    "S-": dist_worst,
    "Score": performance_score,
    "Rank": performance_score.rank(ascending=False)
})
df_results


,S+,S-,Score,Rank
AWS,0.036247,0.111974,0.755453,1.0
Google,0.036786,0.091690,0.713673,2.0
Azure,0.076056,0.051278,0.402704,3.0
IBM,0.123027,0.003542,0.027983,4.0


In [6]:
n = AHP_matrix.shape[0]
lambda_max = (AHP_matrix.dot(weights) / weights).mean()
CI = (lambda_max - n) / (n - 1)
RI = {1:0.0, 2:0.0, 3:0.58, 4:0.90, 5:1.12}[n]  # Random index table
CR = CI / RI
print(f"Consistency Ratio = {CR:.3f}")


Consistency Ratio = 0.044
